In [1]:

# CELL 1: Import Libraries
# ============================================
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"   Pandas version: {pd.__version__}")
print(f"   Numpy version: {np.__version__}")

✅ Libraries imported successfully
   Pandas version: 2.2.2
   Numpy version: 1.26.4


In [2]:

# : Define File Paths
# ====================

# Input file path (YAHAN APNA PATH DAALO)
input_file = r"C:\Users\ACER\Downloads\netflix_customer_churn.csv"

# Output folder
output_folder = r"C:\Users\ACER\Desktop\OTT_Churn_Project\data"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

print(f"📂 Input file: {input_file}")
print(f"📂 Output folder: {output_folder}")

# Check if input file exists
if os.path.exists(input_file):
    print("✅ Input file found!")
else:
    print("❌ ERROR: Input file not found!")
    print("   Please check the path and try again")

📂 Input file: C:\Users\ACER\Downloads\netflix_customer_churn.csv
📂 Output folder: C:\Users\ACER\Desktop\OTT_Churn_Project\data
✅ Input file found!


In [3]:

# : Load Raw Data
# ============================================

# Load the CSV file
df = pd.read_csv(input_file)

# Basic info
print(f"📊 Dataset Shape: {df.shape}")
print(f"   Rows: {df.shape[0]:,}")
print(f"   Columns: {df.shape[1]}")

# First 5 rows
print("\n👀 First 5 rows:")
df.head()

📊 Dataset Shape: (5000, 14)
   Rows: 5,000
   Columns: 14

👀 First 5 rows:


,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,a9b75100-82a8-427a-a208-72f24052884a,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action
1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi
2,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,1.48,Drama
3,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,0.35,Horror
4,4e265c34-103a-4dbb-9553-76c9aa47e946,56,Other,Standard,1.89,13,Africa,Mobile,13.99,1,Crypto,2,0.13,Action


In [4]:

# : Missing Values Check
# ============================================

missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percentage': missing_pct.values.round(2)
})

print("🔍 MISSING VALUES:")
print(missing_df[missing_df['Missing'] > 0])

🔍 MISSING VALUES:
Empty DataFrame
Columns: [Column, Missing, Percentage]
Index: []


In [6]:

#  Clean Customer IDs
# ============================================

# Check for duplicate customer_ids
duplicate_ids = df['customer_id'].duplicated().sum()
print(f"Duplicate customer_ids: {duplicate_ids}")

# Remove duplicates if any
if duplicate_ids > 0:
    df = df.drop_duplicates(subset=['customer_id'])
    print(f"✅ Removed duplicates. New shape: {df.shape}")

# Check customer_id format
print("\n📊 Sample customer_ids:")
print(df['customer_id'].head())
print(f"\n✅ All customer_ids are unique: {df['customer_id'].is_unique}")

Duplicate customer_ids: 0

📊 Sample customer_ids:
0    a9b75100-82a8-427a-a208-72f24052884a
1    49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b
2    4d71f6ce-fca9-4ff7-8afa-197ac24de14b
3    d3c72c38-631b-4f9e-8a0e-de103cad1a7d
4    4e265c34-103a-4dbb-9553-76c9aa47e946
Name: customer_id, dtype: object

✅ All customer_ids are unique: True


In [7]:

# 5: Clean Age Column
# ============================================

print("📊 Age Statistics Before Cleaning:")
print(f"Min: {df['age'].min()}")
print(f"Max: {df['age'].max()}")
print(f"Mean: {df['age'].mean():.1f}")
print(f"Median: {df['age'].median()}")

# Check for invalid ages (below 18 or above 100)
invalid_age = df[(df['age'] < 18) | (df['age'] > 100)]
print(f"\nInvalid ages found: {len(invalid_age)}")

# Replace invalid ages with median
median_age = df['age'].median()
df.loc[(df['age'] < 18) | (df['age'] > 100), 'age'] = median_age

print(f"\n✅ Age column cleaned")
print(f"Now - Min: {df['age'].min()}, Max: {df['age'].max()}")

📊 Age Statistics Before Cleaning:
Min: 18
Max: 70
Mean: 43.8
Median: 44.0

Invalid ages found: 0

✅ Age column cleaned
Now - Min: 18, Max: 70


In [8]:

# : Clean Gender Column
# ============================================

print("📊 Gender Distribution Before:")
print(df['gender'].value_counts(dropna=False))

# Fill missing with 'Unknown'
df['gender'] = df['gender'].fillna('Unknown')

# Standardize values
df['gender'] = df['gender'].replace({
    'M': 'Male',
    'F': 'Female',
    'm': 'Male',
    'f': 'Female',
    'male': 'Male',
    'female': 'Female'
})

print("\n✅ Gender column cleaned")
print("\n📊 Gender Distribution After:")
print(df['gender'].value_counts())

📊 Gender Distribution Before:
gender
Female    1711
Male      1654
Other     1635
Name: count, dtype: int64

✅ Gender column cleaned

📊 Gender Distribution After:
gender
Female    1711
Male      1654
Other     1635
Name: count, dtype: int64


In [9]:

# : Clean Subscription Type
# ============================================

print("📊 Subscription Types Before:")
print(df['subscription_type'].value_counts(dropna=False))

# Fill missing with 'Basic'
df['subscription_type'] = df['subscription_type'].fillna('Basic')

# Standardize
df['subscription_type'] = df['subscription_type'].replace({
    'basic': 'Basic',
    'standard': 'Standard',
    'premium': 'Premium',
    'BASIC': 'Basic',
    'STANDARD': 'Standard',
    'PREMIUM': 'Premium'
})

print("\n✅ Subscription Type cleaned")
print("\n📊 Subscription Types After:")
print(df['subscription_type'].value_counts())

📊 Subscription Types Before:
subscription_type
Premium     1693
Basic       1661
Standard    1646
Name: count, dtype: int64

✅ Subscription Type cleaned

📊 Subscription Types After:
subscription_type
Premium     1693
Basic       1661
Standard    1646
Name: count, dtype: int64


In [10]:

# : Clean Watch Hours
# ============================================

print("📊 Watch Hours Before Cleaning:")
print(f"Min: {df['watch_hours'].min()}")
print(f"Max: {df['watch_hours'].max()}")
print(f"Mean: {df['watch_hours'].mean():.2f}")

# Check for outliers (over 168 hours = 24*7)
outliers = df[df['watch_hours'] > 168]
print(f"\nOutliers (>168 hours): {len(outliers)}")

# Cap extreme values at 99th percentile
cap_value = df['watch_hours'].quantile(0.99)
df['watch_hours'] = df['watch_hours'].clip(upper=cap_value)

# Fill negative values with 0
df.loc[df['watch_hours'] < 0, 'watch_hours'] = 0

print(f"\n✅ Watch Hours cleaned")
print(f"Now - Min: {df['watch_hours'].min()}, Max: {df['watch_hours'].max():.1f}, Mean: {df['watch_hours'].mean():.2f}")

📊 Watch Hours Before Cleaning:
Min: 0.01
Max: 110.4
Mean: 11.65

Outliers (>168 hours): 0

✅ Watch Hours cleaned
Now - Min: 0.01, Max: 55.0, Mean: 11.49


In [12]:

#: Clean Last Login Days
# ============================================

print("📊 Last Login Days Before:")
print(f"Min: {df['last_login_days'].min()}")
print(f"Max: {df['last_login_days'].max()}")
print(f"Mean: {df['last_login_days'].mean():.1f}")

# Check for negative values
neg_days = df[df['last_login_days'] < 0]
print(f"\nNegative values: {len(neg_days)}")

# Replace negative with 0
df.loc[df['last_login_days'] < 0, 'last_login_days'] = 0

# Cap at 365 days (1 year)
df['last_login_days'] = df['last_login_days'].clip(upper=365)

print(f"\n✅ Last Login Days cleaned")
print(f"Now - Min: {df['last_login_days'].min()}, Max: {df['last_login_days'].max()}")

📊 Last Login Days Before:
Min: 0
Max: 60
Mean: 30.1

Negative values: 0

✅ Last Login Days cleaned
Now - Min: 0, Max: 60


In [13]:

# : Clean Region
# ============================================

print("📊 Regions Before:")
print(df['region'].value_counts(dropna=False))

# Fill missing with 'Unknown'
df['region'] = df['region'].fillna('Unknown')

# Standardize region names
region_mapping = {
    'NA': 'North America',
    'north america': 'North America',
    'usa': 'North America',
    'uk': 'Europe',
    'eu': 'Europe',
    'asia': 'Asia',
    'africa': 'Africa',
    'oceania': 'Oceania',
    'south america': 'South America'
}
df['region'] = df['region'].replace(region_mapping)

print("\n✅ Region cleaned")
print("\n📊 Regions After:")
print(df['region'].value_counts())

📊 Regions Before:
region
South America    873
Europe           867
North America    851
Asia             841
Africa           803
Oceania          765
Name: count, dtype: int64

✅ Region cleaned

📊 Regions After:
region
South America    873
Europe           867
North America    851
Asia             841
Africa           803
Oceania          765
Name: count, dtype: int64


In [14]:

# : Clean Device
# ============================================

print("📊 Devices Before:")
print(df['device'].value_counts(dropna=False))

# Fill missing with 'Unknown'
df['device'] = df['device'].fillna('Unknown')

# Standardize device names
device_mapping = {
    'mobile': 'Mobile',
    'phone': 'Mobile',
    'smartphone': 'Mobile',
    'tablet': 'Tablet',
    'ipad': 'Tablet',
    'laptop': 'Laptop',
    'pc': 'Desktop',
    'desktop': 'Desktop',
    'tv': 'TV',
    'smart tv': 'TV'
}
df['device'] = df['device'].replace(device_mapping)

print("\n✅ Device cleaned")
print("\n📊 Devices After:")
print(df['device'].value_counts())

📊 Devices Before:
device
Tablet     1048
Laptop     1006
Mobile     1004
TV          993
Desktop     949
Name: count, dtype: int64

✅ Device cleaned

📊 Devices After:
device
Tablet     1048
Laptop     1006
Mobile     1004
TV          993
Desktop     949
Name: count, dtype: int64


In [15]:

# : Clean Monthly Fee
# ============================================

print("📊 Monthly Fee Before:")
print(f"Min: ₹{df['monthly_fee'].min()}")
print(f"Max: ₹{df['monthly_fee'].max()}")
print(f"Mean: ₹{df['monthly_fee'].mean():.2f}")

# Check for invalid fees (below 0 or above 100)
invalid_fees = df[(df['monthly_fee'] < 0) | (df['monthly_fee'] > 100)]
print(f"\nInvalid fees: {len(invalid_fees)}")

# Replace invalid with median by subscription type
for sub_type in df['subscription_type'].unique():
    median_fee = df[df['subscription_type'] == sub_type]['monthly_fee'].median()
    mask = (df['subscription_type'] == sub_type) & ((df['monthly_fee'] < 0) | (df['monthly_fee'] > 100))
    df.loc[mask, 'monthly_fee'] = median_fee

print(f"\n✅ Monthly Fee cleaned")
print(f"Now - Min: ₹{df['monthly_fee'].min()}, Max: ₹{df['monthly_fee'].max()}")

📊 Monthly Fee Before:
Min: ₹8.99
Max: ₹17.99
Mean: ₹13.68

Invalid fees: 0

✅ Monthly Fee cleaned
Now - Min: ₹8.99, Max: ₹17.99


In [16]:

#: Clean Churned Column
# =====================

print("📊 Churned Before:")
print(df['churned'].value_counts(dropna=False))

# Fill missing with 0 (not churned)
df['churned'] = df['churned'].fillna(0)

# Convert to integer
df['churned'] = df['churned'].astype(int)

# Ensure only 0 and 1
df.loc[df['churned'] > 1, 'churned'] = 1
df.loc[df['churned'] < 0, 'churned'] = 0

print("\n✅ Churned column cleaned")
print("\n📊 Churned After:")
print(df['churned'].value_counts())
print(f"Churn Rate: {df['churned'].mean()*100:.1f}%")

📊 Churned Before:
churned
1    2515
0    2485
Name: count, dtype: int64

✅ Churned column cleaned

📊 Churned After:
churned
1    2515
0    2485
Name: count, dtype: int64
Churn Rate: 50.3%


In [17]:

# : Clean Payment Method
# ============================================

print("📊 Payment Methods Before:")
print(df['payment_method'].value_counts(dropna=False))

# Fill missing with 'Unknown'
df['payment_method'] = df['payment_method'].fillna('Unknown')

# Standardize payment methods
payment_mapping = {
    'credit': 'Credit Card',
    'debit': 'Debit Card',
    'card': 'Credit Card',
    'cc': 'Credit Card',
    'paypal': 'PayPal',
    'gift': 'Gift Card',
    'giftcard': 'Gift Card',
    'crypto': 'Crypto',
    'bitcoin': 'Crypto'
}
df['payment_method'] = df['payment_method'].replace(payment_mapping)

print("\n✅ Payment Method cleaned")
print("\n📊 Payment Methods After:")
print(df['payment_method'].value_counts())

📊 Payment Methods Before:
payment_method
Debit Card     1030
PayPal         1026
Crypto          995
Gift Card       976
Credit Card     973
Name: count, dtype: int64

✅ Payment Method cleaned

📊 Payment Methods After:
payment_method
Debit Card     1030
PayPal         1026
Crypto          995
Gift Card       976
Credit Card     973
Name: count, dtype: int64


In [18]:

# : Clean Number of Profiles
# ============================================

print("📊 Number of Profiles Before:")
print(f"Min: {df['number_of_profiles'].min()}")
print(f"Max: {df['number_of_profiles'].max()}")
print(df['number_of_profiles'].value_counts().sort_index())

# Fill missing with 1
df['number_of_profiles'] = df['number_of_profiles'].fillna(1)

# Convert to integer
df['number_of_profiles'] = df['number_of_profiles'].astype(int)

# Cap at reasonable maximum (10 profiles)
df['number_of_profiles'] = df['number_of_profiles'].clip(upper=10)

print(f"\n✅ Number of Profiles cleaned")
print(f"Now - Min: {df['number_of_profiles'].min()}, Max: {df['number_of_profiles'].max()}")

📊 Number of Profiles Before:
Min: 1
Max: 5
number_of_profiles
1     972
2    1001
3     994
4     999
5    1034
Name: count, dtype: int64

✅ Number of Profiles cleaned
Now - Min: 1, Max: 5


In [19]:

# : Clean Average Watch Time
# ============================================

print("📊 Avg Watch Time Before:")
print(f"Min: {df['avg_watch_time_per_day'].min()}")
print(f"Max: {df['avg_watch_time_per_day'].max()}")
print(f"Mean: {df['avg_watch_time_per_day'].mean():.2f}")

# Check for outliers (over 24 hours)
outliers = df[df['avg_watch_time_per_day'] > 24]
print(f"\nOutliers (>24 hours): {len(outliers)}")

# Cap at 24 hours
df['avg_watch_time_per_day'] = df['avg_watch_time_per_day'].clip(upper=24)

# Fill negative with 0
df.loc[df['avg_watch_time_per_day'] < 0, 'avg_watch_time_per_day'] = 0

print(f"\n✅ Avg Watch Time cleaned")
print(f"Now - Min: {df['avg_watch_time_per_day'].min()}, Max: {df['avg_watch_time_per_day'].max()}")

📊 Avg Watch Time Before:
Min: 0.0
Max: 98.42
Mean: 0.87

Outliers (>24 hours): 10

✅ Avg Watch Time cleaned
Now - Min: 0.0, Max: 24.0


In [20]:

# : Clean Favorite Genre
# ============================================

print("📊 Favorite Genres Before:")
print(df['favorite_genre'].value_counts(dropna=False).head(10))

# Fill missing with 'Unknown'
df['favorite_genre'] = df['favorite_genre'].fillna('Unknown')

# Standardize genre names
genre_mapping = {
    'action': 'Action',
    'ACTION': 'Action',
    'comedy': 'Comedy',
    'COMEDY': 'Comedy',
    'drama': 'Drama',
    'DRAMA': 'Drama',
    'horror': 'Horror',
    'HORROR': 'Horror',
    'sci-fi': 'Sci-Fi',
    'scifi': 'Sci-Fi',
    'sci fi': 'Sci-Fi',
    'romance': 'Romance',
    'ROMANCE': 'Romance',
    'documentary': 'Documentary',
    'DOCUMENTARY': 'Documentary'
}
df['favorite_genre'] = df['favorite_genre'].replace(genre_mapping)

print("\n✅ Favorite Genre cleaned")
print("\n📊 Favorite Genres After:")
print(df['favorite_genre'].value_counts().head(10))

📊 Favorite Genres Before:
favorite_genre
Drama          731
Documentary    729
Romance        725
Sci-Fi         720
Horror         713
Action         697
Comedy         685
Name: count, dtype: int64

✅ Favorite Genre cleaned

📊 Favorite Genres After:
favorite_genre
Drama          731
Documentary    729
Romance        725
Sci-Fi         720
Horror         713
Action         697
Comedy         685
Name: count, dtype: int64


In [21]:

# : Final Quality Check
# ============================================

print("🔍 FINAL DATA QUALITY CHECK")
print("="*50)

# 1. No missing values
print(f"\n1. Missing values: {df.isnull().sum().sum()} ✅")

# 2. Correct data types
print("\n2. Data types:")
print(df.dtypes)

# 3. Reasonable ranges
print("\n3. Value ranges:")
print(f"   Age: {df['age'].min()}-{df['age'].max()}")
print(f"   Watch hours: {df['watch_hours'].min():.1f}-{df['watch_hours'].max():.1f}")
print(f"   Last login: {df['last_login_days'].min()}-{df['last_login_days'].max()}")
print(f"   Monthly fee: ₹{df['monthly_fee'].min()}-₹{df['monthly_fee'].max()}")

# 4. Unique customer_ids
print(f"\n4. Unique customer_ids: {df['customer_id'].nunique()} ✅")

🔍 FINAL DATA QUALITY CHECK

1. Missing values: 0 ✅

2. Data types:
customer_id                object
age                         int64
gender                     object
subscription_type          object
watch_hours               float64
last_login_days             int64
region                     object
device                     object
monthly_fee               float64
churned                     int32
payment_method             object
number_of_profiles          int32
avg_watch_time_per_day    float64
favorite_genre             object
dtype: object

3. Value ranges:
   Age: 18-70
   Watch hours: 0.0-55.0
   Last login: 0-60
   Monthly fee: ₹8.99-₹17.99

4. Unique customer_ids: 5000 ✅


In [22]:

# : Replace Customer ID with Short ID
# ============================================

print("🔍 Original Customer ID sample:")
print(df['customer_id'].head())

# Create a new numeric ID column
df['customer_id_numeric'] = range(1, len(df) + 1)

# Create mapping dictionary for reference
id_mapping = dict(zip(df['customer_id'], df['customer_id_numeric']))

print(f"\n✅ Created numeric IDs from 1 to {len(df)}")

# Show mapping sample
print("\n📊 Sample mapping (first 5):")
for old_id, new_id in list(id_mapping.items())[:5]:
    print(f"   {new_id:4d} → {old_id}")

# Keep both old and new IDs (optional)
# df = df.drop('customer_id', axis=1)  # Uncomment if you want to remove old IDs

print("\n📊 Data with new numeric ID:")
print(df[['customer_id_numeric', 'customer_id', 'age', 'gender']].head())

🔍 Original Customer ID sample:
0    a9b75100-82a8-427a-a208-72f24052884a
1    49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b
2    4d71f6ce-fca9-4ff7-8afa-197ac24de14b
3    d3c72c38-631b-4f9e-8a0e-de103cad1a7d
4    4e265c34-103a-4dbb-9553-76c9aa47e946
Name: customer_id, dtype: object

✅ Created numeric IDs from 1 to 5000

📊 Sample mapping (first 5):
      1 → a9b75100-82a8-427a-a208-72f24052884a
      2 → 49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b
      3 → 4d71f6ce-fca9-4ff7-8afa-197ac24de14b
      4 → d3c72c38-631b-4f9e-8a0e-de103cad1a7d
      5 → 4e265c34-103a-4dbb-9553-76c9aa47e946

📊 Data with new numeric ID:
   customer_id_numeric                           customer_id  age  gender
0                    1  a9b75100-82a8-427a-a208-72f24052884a   51   Other
1                    2  49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b   47   Other
2                    3  4d71f6ce-fca9-4ff7-8afa-197ac24de14b   27  Female
3                    4  d3c72c38-631b-4f9e-8a0e-de103cad1a7d   53   Other
4                    5  4

In [23]:

# : Save Cleaned Data
# ============================================

output_folder = r"C:\Users\ACER\Desktop\OTT_Churn_Project\data"
os.makedirs(output_folder, exist_ok=True)

# Save CSV
cleaned_file = os.path.join(output_folder, "churn_cleaned_final.csv")
df.to_csv(cleaned_file, index=False)
print(f"✅ Saved: {cleaned_file}")

# Save Excel for analysis
excel_file = os.path.join(output_folder, "churn_cleaned_final.xlsx")
df.to_excel(excel_file, index=False)
print(f"✅ Saved: {excel_file}")

print(f"\n📊 Final dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"📁 File size: {os.path.getsize(cleaned_file)/1024:.1f} KB")

✅ Saved: C:\Users\ACER\Desktop\OTT_Churn_Project\data\churn_cleaned_final.csv
✅ Saved: C:\Users\ACER\Desktop\OTT_Churn_Project\data\churn_cleaned_final.xlsx

📊 Final dataset: 5000 rows, 15 columns
📁 File size: 562.0 KB
